In [41]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

# Your ScraperAPI Key
SCRAPER_API_KEY = "your_api"

# Function to get HTML content using ScraperAPI
def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }
    params = {"api_key": SCRAPER_API_KEY, "url": url, "country_code": "us","render": "true"}
    response = requests.get("http://api.scraperapi.com", params=params)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch {url} - Status Code: {response.status_code}")
        return None

# Function to extract product details from Amazon
def scrape_amazon(search_term):
    search_url = f"https://www.amazon.com/s?k={search_term.replace(' ', '+')}"
    html = get_html(search_url)

    if html:
        soup = BeautifulSoup(html, "html.parser")

        products = []
        for item in soup.select(".s-result-item"):
            name = item.select_one("h2 span.a-size-medium, h2 span.a-size-base-plus")
            price = item.select_one("span.a-price span.a-offscreen")
            rating = item.select_one("span.a-icon-alt")

            if name and price:
                products.append({
                    "Website": "Amazon",
                    "Name": name.text.strip(),
                    "Price": price.text.strip(),
                    "Rating": rating.text.strip() if rating else "No rating"
                })
        
        return products
    return []

# Function to extract product details from Flipkart
def scrape_flipkart(search_term):
    search_url = f"https://www.flipkart.com/search?q={search_term.replace(' ', '%20')}"
    html = get_html(search_url)

    if html:
        soup = BeautifulSoup(html, "html.parser")

        products = []
        for item in soup.select("div._75nlfW"):
            name = item.select_one("div.KzDlHZ")
            price = item.select_one("div.Nx9bqj _4b5DiR")
            rating = item.select_one("div.XQDdHH")

            if name and price:
                products.append({
                    "Website": "Flipkart",
                    "Name": name.text.strip(),
                    "Price": price.text.strip(),
                    "Rating": rating.text.strip() if rating else "No rating"
                })
        
        return products
    return []

# Main function
def main():
    search_term = input("Enter product to search: ").strip()

    amazon_data = scrape_amazon(search_term)
    time.sleep(1)  # Delay to avoid hitting rate limits
    flipkart_data = scrape_flipkart(search_term)

    all_data = amazon_data + flipkart_data

    if all_data:
        df = pd.DataFrame(all_data)
        filename = f"{search_term.replace(' ', '_')}_products.xlsx"
        df.to_excel(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No products found.")

if __name__ == "__main__":
    main()

Enter product to search: card
No products found.
